# Assignment 4

Deadline: 30.04.2025 12:00 CET

<Add your name, student-id and emal address>

## Prerequisites: Library imports, data load and initialization of the backtest service

In [1]:
# Standard library imports
import os
import sys
import copy
from typing import Optional

# Third party imports
import numpy as np
import pandas as pd

# Add the project root directory to Python path
project_root = os.path.dirname(os.path.dirname(os.getcwd()))   #<Change this path if needed>
src_path = os.path.join(project_root, 'qpmwp-course\\src')    #<Change this path if needed>
sys.path.append(project_root)
sys.path.append(src_path)

# Local modules imports
from helper_functions import load_data_spi, load_pickle
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.optimization import Optimization, Objective, MeanVariance
from optimization.optimization_data import OptimizationData
from optimization.constraints import Constraints
from backtesting.backtest_item_builder_classes import (
    SelectionItemBuilder,
    OptimizationItemBuilder,
)
from backtesting.backtest_item_builder_functions import (
    bibfn_selection_min_volume,
    bibfn_selection_gaps,
    bibfn_return_series,
    bibfn_budget_constraint,
    bibfn_box_constraints,
    bibfn_size_dependent_upper_bounds,
)
from backtesting.backtest_data import BacktestData
from backtesting.backtest_service import BacktestService
from backtesting.backtest import Backtest

# custom imports
from scipy.optimize import minimize
from scipy.optimize import minimize_scalar

In [2]:
# Load data
path_to_data = '../data/'  # <change this to your path to data>

# Load market and jkp data from parquet files
market_data = pd.read_parquet(path = f'{path_to_data}market_data.parquet')

# Instantiate the BacktestData class
# and set the market data and jkp data as attributes
data = BacktestData()
data.market_data = market_data
data.bm_series = load_data_spi(path='../data/')  # <change this if necessary>

In [3]:
# Define rebalancing dates
n_days = 21*3
market_data_dates = market_data.index.get_level_values('date').unique().sort_values(ascending=True)
rebdates = market_data_dates[market_data_dates > '2015-01-01'][::n_days].strftime('%Y-%m-%d').tolist()

In [4]:
# Define the selection item builders.
selection_item_builders = {
    'gaps': SelectionItemBuilder(
        bibfn = bibfn_selection_gaps,
        width = 252*3,
        n_days = 10,
    ),
    'min_volume': SelectionItemBuilder(
        bibfn = bibfn_selection_min_volume,
        width = 252,
        min_volume = 500_000,
        agg_fn = np.median,
    ),
}

# Define the optimization item builders.
optimization_item_builders = {
    'return_series': OptimizationItemBuilder(
        bibfn = bibfn_return_series,
        width = 252*3,
        fill_value = 0,
    ),
    'budget_constraint': OptimizationItemBuilder(
        bibfn = bibfn_budget_constraint,
        budget = 1,
    ),
    'box_constraints': OptimizationItemBuilder(
        bibfn = bibfn_box_constraints,
        upper = 0.1,
    ),
    'size_dep_upper_bounds': OptimizationItemBuilder(
        bibfn = bibfn_size_dependent_upper_bounds,
        small_cap = {'threshold': 300_000_000, 'upper': 0.02},
        mid_cap = {'threshold': 1_000_000_000, 'upper': 0.05},
        large_cap = {'threshold': 10_000_000_000, 'upper': 0.1},
    ),
}

# Initialize the backtest service
bs = BacktestService(
    data = data,
    selection_item_builders = selection_item_builders,
    optimization_item_builders = optimization_item_builders,
    rebdates = rebdates,
)

## 1. Maximum Sharpe Ratio Portfolio

a) 

(6 points)

Complete the `MaxSharpe` class below by implementing your its methods `set_objective` and `solve`.
The `solve` method should implement an iterative algorithm that quickly approximates the "true" maximimum Sharpe ratio portfolio (given the estimates of mean and covariance). This approximation should be done by repeatedly solving a mean-variance optimization problem, where the risk aversion parameter (which scales the covariance matrix) is adjusted in each iteration. The algorithm should terminate after a maximum of 10 iterations. 

In [5]:
class MaxSharpe(Optimization):

    def __init__(self,
                 constraints: Optional[Constraints] = None,
                 covariance: Optional[Covariance] = None,
                 expected_return: Optional[ExpectedReturn] = None,
                 iters: int = 10,
                 risk_aversion: float = 1.0,
                 turnover_penalty: float = 0.0,
                 **kwargs) -> None:
        super().__init__(
            constraints=constraints,
            **kwargs,
        )
        self.covariance = Covariance() if covariance is None else covariance
        self.expected_return = ExpectedReturn() if expected_return is None else expected_return
        self.iters = iters
        self.params['risk_aversion'] = risk_aversion
        self.params['turnover_penalty'] = turnover_penalty

    def set_objective(self, optimization_data: OptimizationData) -> None:
        self.data = optimization_data
        X = optimization_data['return_series']
        self.covmat = self.covariance.estimate(X=X, inplace=False)
        self.mu  = self.expected_return.estimate(X=X, inplace=False)
        self.objective = Objective(
            q = self.mu * -1,
            P = self.covmat * 2 * self.params['risk_aversion'],
        )

        self.og_P = (2 * self.covmat).copy()
        return None

    def solve(self) -> None:
        
        # opimization function
        def mean_var_solver(a):
            self.params['risk_aversion'] = a[0]
            self.objective.coefficients['P'] = self.og_P * a[0]
            super(MaxSharpe, self).solve()
            w = np.array(list(self.results['weights'].values()))
            sharpe = (self.mu @ w) / np.sqrt(w @ self.covmat @ w)
            return sharpe

        sharpe_maximization = minimize(
        fun=lambda x: -1.0*mean_var_solver(x),
        x0=np.array([1.0]),
        bounds=[(1e-2, 1e2)],
        method="Powell",
        tol = 1e-3,
        options={"maxiter": self.iters}
        )

        # getting results
        optimal_ra = (sharpe_maximization.x)[0]
        optimal_sr = -1*sharpe_maximization.fun
        self.params['risk_aversion'] = optimal_ra
        self.objective.coefficients['P'] = self.og_P * optimal_ra
        super().solve()
        optimal_w = self.results['weights']

        # storing results
        self.results = {
            'risk_aversion': optimal_ra,
            'sharpe_ratio': optimal_sr,
            'weights': optimal_w
        }

        return None

b) 

(2 points)

Provide a theoretical or empirical justification that your algorithm converges to the true maximum Sharpe ratio portfolio for the given coefficients of mean and covariance.
Hint: If you want to provide an empirical justification, you can perform an optimization for a single point in time by running the following code.

In [6]:
bs.optimization = MaxSharpe(
    covariance=Covariance(method='pearson'),
    expected_return=ExpectedReturn(method='geometric'),
    solver_name='cvxopt',  # <change this to your preferred solver>
    iters = 1,
    risk_aversion = 1.0
)
bs.prepare_rebalancing('2015-01-02')
bs.optimization.set_objective(bs.optimization_data)
bs.optimization.solve()
bs.optimization.results

{'risk_aversion': np.float64(12.242864960945404),
 'sharpe_ratio': np.float64(0.1492260849428679),
 'weights': {'102': 1.3197470159702042e-07,
  '103': 8.639233211997004e-08,
  '104': 0.019999594043156096,
  '111': 0.01903660928227035,
  '113': 2.6860743901970434e-08,
  '122': 2.3850717024806174e-07,
  '125': 0.019999873710221557,
  '127': 0.019999851361862366,
  '128': 0.019999792065448933,
  '13': 1.8588820515896416e-07,
  '139': 0.019999818041858103,
  '141': 0.019999894039832804,
  '142': 1.181840125173049e-07,
  '148': 0.019999881264611094,
  '149': 1.1418424434259269e-07,
  '154': 1.9558733140954522e-07,
  '159': 0.019999645179776152,
  '161': 0.006993952877425998,
  '169': 0.019999889918141566,
  '175': 0.01999951871734015,
  '176': 0.01999853013965164,
  '177': 9.465400149361487e-08,
  '184': 0.019999705047741637,
  '185': 0.01999959810275169,
  '187': 1.6302354829314428e-07,
  '189': 4.7136364827414804e-07,
  '191': 1.1634902611967339e-06,
  '197': 4.7820449882259e-07,
  '2': 

In [7]:
test_iterations = np.arange(0, 101, 10)
Convergence_df = pd.DataFrame(index=test_iterations, columns=['Sharpe Ratio', 'Risk Aversion'])
for iters in test_iterations:
    bs.optimization = MaxSharpe(
    covariance=Covariance(method='pearson'),
    expected_return=ExpectedReturn(method='geometric'),
    solver_name='cvxopt',  # <change this to your preferred solver>
    iters = iters,
    risk_aversion = 1.0
    )
    bs.prepare_rebalancing('2015-01-02')
    bs.optimization.set_objective(bs.optimization_data)
    bs.optimization.solve()
    results = bs.optimization.results
    Convergence_df.loc[iters, 'Sharpe Ratio'] = results['sharpe_ratio']
    Convergence_df.loc[iters, 'Risk Aversion'] = results['risk_aversion']

Convergence_df



KeyboardInterrupt: 

## 2. Backtest MaxSharpe with Turnover Penalty

(5 points)

Calibrate the turnover penalty parameter such that the backtest of the MaxSharpe strategy displays an annual turnover of roughly 100%.

In [ ]:
rebdates_df = pd.Series(rebdates)
rebdates_cal = rebdates_df[rebdates_df > '2018-01-01']
rebdates_cal = rebdates_cal[rebdates_cal< '2020-01-01'].tolist()


bs_cal = BacktestService(
    data = data,
    selection_item_builders = selection_item_builders,
    optimization_item_builders = optimization_item_builders,
    rebdates = rebdates_cal,
)

# Update the backtest service with a MaxSharpe optimization object
bs_cal.optimization = MaxSharpe(
    covariance=Covariance(method='pearson'),
    expected_return=ExpectedReturn(method='geometric'),
    solver_name='cvxopt',    # <change this to your preferred solver>
    turnover_penalty=None,   # <change this>
)

# Instantiate the backtest object
bt_ms = Backtest()

# Run the backtest
bt_ms.run(bs = bs_cal)

['2018-02-22', '2018-05-22', '2018-08-17', '2018-11-14', '2019-02-11', '2019-05-09', '2019-08-06', '2019-11-01']
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
Rebalancing date: 2019-02-11
Rebalancing date: 2019-05-09
Rebalancing date: 2019-08-06
Rebalancing date: 2019-11-01


In [57]:
# getting subset of rebdates to test on
rebdates_df = pd.Series(rebdates)
rebdates_cal = rebdates_df[rebdates_df > '2018-01-01']
rebdates_cal = rebdates_cal[rebdates_cal< '2020-01-01'].tolist()

# creating backtest service with shorter lookback
bs_cal = BacktestService(
    data = data,
    selection_item_builders = selection_item_builders,
    optimization_item_builders = optimization_item_builders,
    rebdates = rebdates_cal,
)

# Update the backtest service with a MaxSharpe optimization object
bs_cal.optimization = MaxSharpe(
    covariance=Covariance(method='pearson'),
    expected_return=ExpectedReturn(method='geometric'),
    solver_name='cvxopt',    # <change this to your preferred solver>
    turnover_penalty=None,   # <change this>
)


def turnover_error(to):
    bs_cal.optimization.params['turnover_penalty'] = to

    bt_ms = Backtest()
    bt_ms.run(bs=bs_cal)

    annual_to = bt_ms.strategy.turnover(return_series = bs_cal.data.get_return_series()).mean() * 4 # 4 is number of rebdates per year

    return abs(annual_to - 1.0)


# callibration
max_iterations = 10

turnover_penalty = minimize_scalar(
    turnover_error,
    bounds=(1e-5, 1e-3),
    method="bounded",
    options={"maxiter": max_iterations, "xatol": 0.0001}
        )

optimal_to_penalty = turnover_penalty.x
achieved_turnover = turnover_penalty.fun

print(optimal_to_penalty)
print(achieved_turnover)

Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
Rebalancing date: 2019-02-11
Rebalancing date: 2019-05-09
Rebalancing date: 2019-08-06
Rebalancing date: 2019-11-01
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
Rebalancing date: 2019-02-11
Rebalancing date: 2019-05-09
Rebalancing date: 2019-08-06
Rebalancing date: 2019-11-01
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
Rebalancing date: 2019-02-11
Rebalancing date: 2019-05-09
Rebalancing date: 2019-08-06
Rebalancing date: 2019-11-01
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
Rebalancing date: 2019-02-11
Rebalancing date: 2019-05-09
Rebalancing date: 2019-08-06
Rebalancing date: 2019-11-01
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing da

In [58]:
# Update the backtest service with a MaxSharpe optimization object
bs.optimization = MaxSharpe(
    covariance=Covariance(method='pearson'),
    expected_return=ExpectedReturn(method='geometric'),
    solver_name='cvxopt',    # <change this to your preferred solver>
    turnover_penalty=optimal_to_penalty,   # <change this>
)

# Instantiate the backtest object
bt_ms = Backtest()

# Run the backtest
bt_ms.run(bs = bs)

annual_to_best = bt_ms.strategy.turnover(return_series = bs.data.get_return_series()).mean() * 4
annual_to_best

Rebalancing date: 2015-01-02
Rebalancing date: 2015-04-01
Rebalancing date: 2015-06-29
Rebalancing date: 2015-09-24
Rebalancing date: 2015-12-22
Rebalancing date: 2016-03-18
Rebalancing date: 2016-06-15
Rebalancing date: 2016-09-12
Rebalancing date: 2016-12-08
Rebalancing date: 2017-03-07
Rebalancing date: 2017-06-02
Rebalancing date: 2017-08-30
Rebalancing date: 2017-11-27
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
Rebalancing date: 2019-02-11
Rebalancing date: 2019-05-09
Rebalancing date: 2019-08-06
Rebalancing date: 2019-11-01
Rebalancing date: 2020-01-29
Rebalancing date: 2020-04-27
Rebalancing date: 2020-07-23
Rebalancing date: 2020-10-20
Rebalancing date: 2021-01-15
Rebalancing date: 2021-04-14
Rebalancing date: 2021-07-12
Rebalancing date: 2021-10-07
Rebalancing date: 2022-01-04
Rebalancing date: 2022-04-01
Rebalancing date: 2022-06-29
Rebalancing date: 2022-09-26
Rebalancing date: 2022-12-22
Rebalancing da

np.float64(0.6811327832789126)

In [9]:
# getting subset of rebdates to test on
rebdates_df = pd.Series(rebdates)
rebdates_cal = rebdates_df[rebdates_df > '2018-01-01']
rebdates_cal = rebdates_cal[rebdates_cal< '2019-01-01'].tolist()

# creating backtest service with shorter lookback
bs = BacktestService(
    data = data,
    selection_item_builders = selection_item_builders,
    optimization_item_builders = optimization_item_builders,
    rebdates = rebdates_cal,
)

In [ ]:
bs.optimization = MaxSharpe(
        covariance=Covariance(method='pearson'),
        expected_return=ExpectedReturn(method='geometric'),
        solver_name='cvxopt',    # <change this to your preferred solver>
        turnover_penalty=None,
        )

tol = 0.05
to_array = np.linspace(1e-5, 1e-3, 5)
print(to_array)
errors = []

best_to = 0
best_annual_to = 0

for i in range(len(to_array)):
    to = to_array[i]
    bs.optimization.params['turnover_penalty'] = to
    
    # Instantiate the backtest object
    bt_ms = Backtest()

    # Run the backtest
    bt_ms.run(bs = bs)

    annual_to = bt_ms.strategy.turnover(return_series = bs.data.get_return_series()).mean() * 4

    errors.append(abs(annual_to - 1.0))
    print(f'for index {i} and to {to} we have annual turnover {annual_to} with error {errors[i]}')

    if errors[i] < tol:
        best_to = to_array[i]
        best_annual_to = annual_to
        break

    elif (i>1) and (errors[i]>errors[i-1]):
        new_to_array = np.linspace(to_array[i-2], to_array[i], 6)[:-1]
        print('our new array is', new_to_array)
        new_errors = []
        for j in range(len(new_to_array)):
            new_to = new_to_array[j]
            bs.optimization.params['turnover_penalty'] = new_to

            # Instantiate the backtest object
            bt_ms = Backtest()

            # Run the backtest
            bt_ms.run(bs = bs)

            new_annual_to = bt_ms.strategy.turnover(return_series = bs.data.get_return_series()).mean() * 4

            new_errors.append(abs(annual_to - 1.0))

            print(f'for index {j} and to {new_to} we have annual turnover {new_annual_to} with error {new_errors[i]}')

            if new_errors[j] < tol:
                best_to = new_to_array[j]
                best_annual_to = new_annual_to
                break
            elif (j>0) and (new_errors[j]>new_errors[j-1]):
                best_to = new_errors[j-1]
                best_annual_to = new_annual_to
                break

        best_to = new_to_array[-1]
        best_annual_to = annual_to
        break
            
print(best_to)
print(best_annual_to)

[1.000e-05 2.575e-04 5.050e-04 7.525e-04 1.000e-03]
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
for index 0 and to 1e-05 we have annual turnover 1.9599073302981607 with error 0.9599073302981607
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
for index 1 and to 0.0002575 we have annual turnover 1.5501903170878277 with error 0.5501903170878277
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
for index 2 and to 0.000505 we have annual turnover 1.4311671607733938 with error 0.4311671607733938
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22
Rebalancing date: 2018-08-17
Rebalancing date: 2018-11-14
for index 3 and to 0.0007525 we have annual turnover 1.3953205479919784 with error 0.39532054799197835
Rebalancing date: 2018-02-22
Rebalancing date: 2018-05-22


## 3. Simulation and Descriptive Statistics

(3 points)

- Simulate the portfolio returns from your MaxSharpe backtest. Use fixed costs of 1% and variable costs of 0.3%.
- Plot the cumulated returns of the MaxSharpe strategy together with those of the SPI Index.
- Plot the turnover of your MaxSharpe strategy over time.
- Print the annualized turnover (computed as the average turnover over the backtest multiplied by the number of rebalancing per year) for your MaxSharpe strategy.
- Create and print a table with descriptive performance statistics for your MaxSharpe strategy and the SPI Index.
